# Proportional-Integral Controller

In [ ]:
from tuto_control_lib.systems import IntroSystem
from tuto_control_lib.plot import *

import matplotlib.pyplot as plt
import numpy as np
from math import exp, log, pi, cos
from statistics import mean

As we have seen before, a Proportional controller is inheritly imprecise.

One way to improve the precision of the closed loop system is to add an integral term to the controller.

The integral term aims at canceling the steady state error.

The form of the controller (in discrete time) is the following:

$$
u(k) = K_p e(k) + K_i \sum_{i=0}^k e(i)
$$

We can try to add the $K_i$ term to the previously defined P Controller:

In [ ]:
max_iter = 50
reference_value = 1
kp = 2.5
ki = 1.5
y_values, u_values, u, system, integral = [], [], 0, IntroSystem(), 0

for _ in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    error = reference_value - y
    integral += error
    u = kp * error + ki * integral
    
    system.apply(u)
    u_values.append(u)

plot_u_y(u_values, y_values, reference_value)

We can see that the system converges to the reference value!
However, there are some oscillations and overshooting...

<div class="alert alert-info" role="alert">
  Try to change the values of $K_p$ and $K_i$ to observe the change of behavior.
</div>

# Design of a PI Controller

As for the P Controller, we have to chose the desired closed loop behavior.

In the case of a PI Controller, we have the precision by the integral term, and the precision as for the P Controller.

There are several methods to find gains for a PI Controller.
In the following we use the *pole placement method*.
The idea is to chose the poles of the closed-loop system to fit the desired behavior.

Without too much details to avoid being too "mathy", we give the equations leading to the gains.

Given the desired values for $k_s$ (settling time) and $M_p$ (max. overshoot), we get:

$$
\begin{cases}
K_p = \frac{a - r^2}{b}\\
K_i = \frac{1 - 2 r \cos \theta + r^2}{b}
\end{cases}
$$

with:

- $r = \exp\left(-\frac{4}{k_s}\right)$
- $\theta = \pi \frac{\log r}{\log M_p}$

In our case:

In [ ]:
# The coefficients of our system
a = 0.8
b = 0.5

# Our desired properties
ks = 10
mp = 0.01

r = exp(-4/ks)
theta = pi * log(r) / log(mp)

kp = (a - r * r) / b
ki = (1 - 2 * r * cos(theta) + r * r) / b

print(f"Kp = {kp}\nKi = {ki}")

In [ ]:
max_iter = 50
reference_value = 1
y_values, u_values, u, system, integral_error = [], [], 0, IntroSystem(), 0

for i in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    error = reference_value - y
    integral_error += error
    u = kp * error + ki * integral_error
    
    system.apply(u)
    u_values.append(u)

plot_u_y(u_values, y_values, reference_value)

In [ ]:
e_ss = reference_value - y_values[-1]
max_overshoot = (max(y_values) - y_values[-1]) / y_values[-1]
settling_time = len([x for x in y_values if abs(x - y_values[-1]) > 0.05])

print(f"Precision: {e_ss}")
print(f"Settling Time: {settling_time} -> desired: < {ks}")
print(f"Max. Overshoot: {max_overshoot} -> desired: < {mp}")

<div class="alert alert-info" role="alert">
  Try to change the requirements on the closed-loop properties to find different values of $K_p$ and $K_i$ and plot the system.
</div>

[Back to menu](./00_Main.ipynb) or [Next chapter](./05_Identification.ipynb)